In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-tes-02-marzo/train_temporal.csv
/kaggle/input/train-tes-02-marzo/test_temporal.csv


In [15]:
!pip install prophet --no-cache-dir
!pip install --no-cache-dir plotly
!pip install cmdstanpy --no-cache-dir
!pip install holidays --no-cache-dir
!pip install --no-cache-dir --upgrade ipywidgets

In [16]:
import pickle
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from prophet.diagnostics import cross_validation, performance_metrics
from holidays import Spain

In [17]:
# Cargar los datos desde los archivos en Kaggle
df_entrenamiento = pd.read_csv("/kaggle/input/train-tes-02-marzo/train_temporal.csv")
df_prueba = pd.read_csv("/kaggle/input/train-tes-02-marzo/test_temporal.csv")

In [18]:
# Convertir la columna de fecha a datetime y renombrar columnas
df_entrenamiento["ds"] = pd.to_datetime(df_entrenamiento["fecha"])
df_entrenamiento = df_entrenamiento.rename(columns={"valor_(GWh)": "y"})

df_prueba["ds"] = pd.to_datetime(df_prueba["fecha"])
df_prueba = df_prueba.rename(columns={"valor_(GWh)": "y"})

In [19]:
# Obtener los días festivos en España

# Se define el rango de años a considerar en base a los datos de entrenamiento.
# Esto asegura que los festivos cubran todo el período de análisis y unos años adicionales para predicción.
years_range = range(df_entrenamiento["ds"].dt.year.min(), df_entrenamiento["ds"].dt.year.max() + 3)

# Se obtiene la lista de festivos de España dentro del rango de años definido.
# 'holidays.Spain' genera un diccionario donde las claves son fechas y los valores son nombres de festivos.
holidays_spain = Spain(years=years_range)


# Crear un set con las fechas de festivos
# Convertimos las claves (fechas) de 'holidays_spain' en una lista de fechas en formato datetime.
# Luego, las almacenamos en un conjunto (set) para facilitar las búsquedas rápidas.
festivos_set = set(pd.to_datetime([pd.Timestamp(date) for date in holidays_spain.keys()]))

# Crear la variable exógena binaria 'es_festivo'
# Para cada fecha en el dataset de entrenamiento, verificamos si está en 'festivos_set'.
# Si la fecha es un festivo, asignamos un 1; si no, un 0. Esto crea una variable binaria para Prophet.
df_entrenamiento["es_festivo"] = df_entrenamiento["ds"].apply(lambda x: 1 if x in festivos_set else 0)

# Se repite el mismo proceso para el dataset de prueba.
# Esto garantiza que 'es_festivo' esté disponible en ambos conjuntos (entrenamiento y validación).
df_prueba["es_festivo"] = df_prueba["ds"].apply(lambda x: 1 if x in festivos_set else 0)


In [20]:
#  Prueba con distintas granularidades
# Se definen los intervalos de tiempo en los que se desea hacer la predicción.
# Cada clave representa un período de agregación y su valor indica la frecuencia y la cantidad de períodos a predecir.

granularidades = {
    "Día a día": ("D", 30),
    "Semana a semana": ("W", 12),
    "Mes a mes": ("ME", 12),
    "Trimestre a trimestre": ("QE", 8), 
    "Semestre a semestre": ("2Q", 6),
    "Año a año": ("YE", 3)
}

In [21]:
# Función para agrupar datos según la granularidad seleccionada
def agrupar_datos(df, unidad):
    df_agrupado = df.groupby(pd.Grouper(key="ds", freq=unidad)).agg({
        "y": "mean",  # Promediar la variable objetivo
        "es_festivo": "mean"  # Mantener es_festivo como promedio (0 a 1)
    }).reset_index()
    
    return df_agrupado

# Antes de entrenar Prophet, debemos asegurarnos de que la variable es_festivo se mantenga en todas las granularidades. Para esto: Promediaremos es_festivo en la granularidad correspondiente para que siempre esté presente.
# Nos aseguraremos de que df_agrupado contenga es_festivo antes de entrenar Prophet.
# Esta agrupación determina la ondulación de las predicciones por granularides, dado que reduce el numero de filas por los promedios de Y para cada periodo

In [22]:
# Entrenar y evaluar Prophet en cada granularidad
modelos_granulares = {}
predicciones_granulares = {}

# Se recorre el diccionario 'granularidades', donde 'nombre' es la descripción 
# (por ejemplo, "Día a día") y 'unidad' es la frecuencia de tiempo ("D", "W", etc.).
# 'periodo' representa la cantidad de períodos a predecir en cada granularidad.

for nombre, (unidad, periodo) in granularidades.items():
    print(f"Entrenando modelo con granularidad: {nombre}")


        # Se llama a la función 'agrupar_datos()' para transformar los datos de entrenamiento
    # a la frecuencia temporal correspondiente (día, semana, mes, etc.).
    
    # Agrupar datos asegurando que 'es_festivo' esté presente
    df_agrupado = agrupar_datos(df_entrenamiento, unidad)

        # Al agrupar los datos, algunas granularidades pueden perder la columna 'es_festivo'.
    # Si la columna no existe después de la agregación, la creamos y le asignamos 0,
    # asumiendo que no hay información de festivos en ese nivel de agregación.
    
    # Comprobar que es_festivo sigue en df_agrupado
    if "es_festivo" not in df_agrupado.columns:
        df_agrupado["es_festivo"] = 0  # Asignar 0 si falta

    # Ajustar Prophet con la variable exógena
    modelo = Prophet()
    modelo.add_regressor("es_festivo")

    # Prophet ajusta sus parámetros en función de los valores de 'y' y la influencia de 'es_festivo'
    
    modelo.fit(df_agrupado)
    
    # Generar predicciones d,s,m,s,a
    df_futuro = modelo.make_future_dataframe(periods=periodo, freq=unidad)
    
    # Asegurar que 'es_festivo' también esté en df_futuro
    df_futuro["es_festivo"] = df_futuro["ds"].apply(lambda x: 1 if x in festivos_set else 0)

    predicciones = modelo.predict(df_futuro)

   
    # Guardar modelo y predicciones
    modelos_granulares[nombre] = modelo
    predicciones_granulares[nombre] = predicciones

    # Graficar resultados para cada granularidad
    fig = go.Figure()

    # Datos históricos
    fig.add_trace(go.Scatter(
        x=df_agrupado["ds"], y=df_agrupado["y"], 
        mode="markers", name="Datos Históricos",
        marker=dict(color="black", opacity=0.5)
    ))

    # Predicción
    fig.add_trace(go.Scatter(
        x=predicciones["ds"], y=predicciones["yhat"], 
        mode="lines", name="Predicción",
        line=dict(color="blue")
    ))

    fig.update_layout(
        title=f"Predicción con granularidad: {nombre}",
        xaxis_title="Fecha",
        yaxis_title="Demanda en GWh",
        template="plotly_white"
    )

    fig.show()

14:55:24 - cmdstanpy - INFO - Chain [1] start processing


Entrenando modelo con granularidad: Día a día


14:55:24 - cmdstanpy - INFO - Chain [1] done processing


14:55:25 - cmdstanpy - INFO - Chain [1] start processing
14:55:25 - cmdstanpy - INFO - Chain [1] done processing


Entrenando modelo con granularidad: Semana a semana


...

<ipython-input-21-47ae6450f785>:3: FutureWarning:

'Q' is deprecated and will be removed in a future version, please use 'QE' instead.

14:55:26 - cmdstanpy - INFO - Chain [1] start processing
14:55:26 - cmdstanpy - INFO - Chain [1] done processing


Entrenando modelo con granularidad: Semestre a semestre


/usr/local/lib/python3.10/dist-packages/prophet/forecaster.py:1854: FutureWarning:

'Q' is deprecated and will be removed in a future version, please use 'QE' instead.



14:55:26 - cmdstanpy - INFO - Chain [1] start processing
14:55:26 - cmdstanpy - INFO - Chain [1] done processing


Entrenando modelo con granularidad: Año a año


In [23]:
print("Valores originales de y (antes de entrenar Prophet):")
print(df_entrenamiento["y"].head())  # Verifica los primeros valores

Valores originales de y (antes de entrenar Prophet):
0    0.267689
1    0.349814
2    0.714860
3    0.788024
4    0.719847
Name: y, dtype: float64


In [24]:
# Evaluar modelo en conjunto de prueba
modelo_final = modelos_granulares["Día a día"]  # Se usa el modelo con granularidad diaria
predicciones_prueba = modelo_final.predict(df_prueba)

y_real = df_prueba["y"].values
y_pred = predicciones_prueba["yhat"].values

mae = mean_absolute_error(y_real, y_pred)
rmse = np.sqrt(mean_squared_error(y_real, y_pred))
r2 = r2_score(y_real, y_pred)

print(f"Evaluación del modelo diario en el conjunto de prueba:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")

Evaluación del modelo diario en el conjunto de prueba:
MAE: 0.07
RMSE: 0.09
R²: 0.7031


In [25]:
# Asegurar frecuencia diaria en df_entrenamiento
df_entrenamiento = df_entrenamiento.set_index("ds").asfreq("D").reset_index()

# Entrenar modelo Prophet
modelo = Prophet()
modelo.add_regressor("es_festivo")
modelo.fit(df_entrenamiento)

# Generar predicciones futuras
df_futuro = modelo.make_future_dataframe(periods=len(df_prueba), freq="D")
df_futuro["es_festivo"] = df_futuro["ds"].apply(lambda x: 1 if x in festivos_set else 0)

# Predicciones
predicciones = modelo.predict(df_futuro)
predicciones_test = predicciones[predicciones["ds"].isin(df_prueba["ds"])]

# Convertir fechas a datetime
df_prueba.loc[:, "ds"] = pd.to_datetime(df_prueba["ds"])
predicciones_test.loc[:, "ds"] = pd.to_datetime(predicciones_test["ds"])

# Graficar
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_prueba["ds"], y=df_prueba["y"], mode="lines", name="Datos Reales", line=dict(color="black")))
fig.add_trace(go.Scatter(x=predicciones_test["ds"], y=predicciones_test["yhat"], mode="lines", name="Predicción", line=dict(color="blue")))

fig.update_layout(title="Comparación de Predicciones vs Datos Reales", xaxis_title="Fecha", yaxis_title="Demanda en GWh", legend_title="Elementos")
fig.show()

14:55:26 - cmdstanpy - INFO - Chain [1] start processing
14:55:27 - cmdstanpy - INFO - Chain [1] done processing


In [26]:
import pickle

#  Guardar todos los modelos y sus predicciones respetando la estructura de granularidades
modelo_guardado = {
    "modelos_granulares": modelos_granulares,  #  Diccionario con todos los modelos entrenados por granularidad
    "predicciones_granulares": predicciones_granulares,  #  Predicciones de cada granularidad
}

# Guardar cada modelo Prophet en archivos .pkl separados según su granularidad
for nombre, modelo in modelos_granulares.items():
    ruta_guardado = f"/kaggle/working/modelo_prophet_{nombre}.pkl"
    with open(ruta_guardado, "wb") as f:
        pickle.dump(modelo, f)
    print(f"Modelo Prophet para {nombre} guardado en {ruta_guardado}")


print(f"Modelos por granularidad guardados en {ruta_guardado}")




Modelo Prophet para Día a día guardado en /kaggle/working/modelo_prophet_Día a día.pkl
Modelo Prophet para Semana a semana guardado en /kaggle/working/modelo_prophet_Semana a semana.pkl
Modelo Prophet para Mes a mes guardado en /kaggle/working/modelo_prophet_Mes a mes.pkl
Modelo Prophet para Trimestre a trimestre guardado en /kaggle/working/modelo_prophet_Trimestre a trimestre.pkl
Modelo Prophet para Semestre a semestre guardado en /kaggle/working/modelo_prophet_Semestre a semestre.pkl
Modelo Prophet para Año a año guardado en /kaggle/working/modelo_prophet_Año a año.pkl
Modelos por granularidad guardados en /kaggle/working/modelo_prophet_Año a año.pkl
